In [19]:
import pandas as pd
import numpy as np
import math
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)

In [20]:
train_raw = pd.read_csv('./Data/training_set_values.csv')
test_raw = pd.read_csv('./Data/test_set_values.csv')

train_raw['train'] = 1
test_raw['train'] = 0
data = pd.concat([train_raw, test_raw])

In [22]:
# Overview

#data.info()

In [21]:
# Identify missing values in numerical data

int_var = ['population','gps_height','num_private','construction_year']
float_var = ['amount_tsh','longitude']

for var in int_var:
    print('{}:'.format(var))
    display(data[var].min())
    display(len(data[data[var] == 0]))

for var in float_var:
    print('{}:'.format(var))
    display(data[var].min())
    display(len(data[data[var] == 0.0]))

print('latitude:')
display(data['latitude'].min())
display(len(data[(data['latitude'] > -0.001) & (data['latitude'] < 0.001)]))

population:


0

26834

gps_height:


-90

...

0.0

2269

latitude:


-11.64944018

2269

In [22]:
# Replace zeros by NaN

for var in int_var:
    data[var].replace(0, np.nan, inplace=True)
    
for var in float_var:
    data[var].replace(0.0, np.nan, inplace=True)

data['latitude'].where((data['latitude'] < -0.001) | (data['latitude'] > 0.001), other= np.nan, inplace=True,axis=0)

train = data[data['train'] == 1]
test = data[data['train'] == 0]

In [23]:
# Duplicate critical columnns for imputation based on normal distribution and random choice

null_features = ['longitude','latitude','gps_height','population','construction_year','amount_tsh']
#no calculations for num_private since they are dropped later (too many missing values)

for null_feature in null_features:
    data['_'.join([null_feature, 'imp_normal'])] = data[null_feature]
    data['_'.join([null_feature, 'imp_random_choice'])] = data[null_feature]

In [24]:
data

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,train,longitude_imp_normal,longitude_imp_random_choice,latitude_imp_normal,latitude_imp_random_choice,gps_height_imp_normal,gps_height_imp_random_choice,population_imp_normal,population_imp_random_choice,construction_year_imp_normal,construction_year_imp_random_choice,amount_tsh_imp_normal,amount_tsh_imp_random_choice
0,69572,6000.0,2011-03-14,Roman,1390.0,Roman,34.938093,-9.856322,none,NaN,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109.0,True,GeoData Consultants Ltd,VWC,Roman,False,1999.0,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1,34.938093,34.938093,-9.856322,-9.856322,1390.0,1390.0,109.0,109.0,1999.0,1999.0,6000.0,6000.0
1,8776,NaN,2013-03-06,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,NaN,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280.0,NaN,GeoData Consultants Ltd,Other,NaN,True,2010.0,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1,34.698766,34.698766,-2.147466,-2.147466,1399.0,1399.0,280.0,280.0,2010.0,2010.0,NaN,NaN
2,34310,25.0,2013-02-25,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,NaN,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250.0,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009.0,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1,37.460664,37.460664,-3.821329,-3.821329,686.0,686.0,250.0,250.0,2009.0,2009.0,25.0,25.0
3,67743,NaN,2013-01-28,Unicef,263.0,UNICEF,38.486161,-11.155298,Zahanati Ya Nanyumbu,NaN,Ruvuma / Southern Coast,Mahakamani,Mtwara,90,63,Nanyumbu,Nanyumbu,58.0,True,GeoData Consultants Ltd,VWC,NaN,True,1986.0,submersible,submersible,submersible,vwc,user-group,never pay,never pay,soft,good,dry,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,1,38.486161,38.486161,-11.155298,-11.155298,263.0,263.0,58.0,58.0,1986.0,1986.0,NaN,NaN
4,19728,NaN,2011-07-13,Action In A,NaN,Artisan,31.130847,-1.825359,Shuleni,NaN,Lake Victoria,Kyanyamisa,Kagera,18,1,Karagwe,Nyakasimbi,NaN,True,GeoData Consultants Ltd,NaN,NaN,True,NaN,gravity,gravity,gravity,other,other,never pay,never pay,soft,good,seasonal,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1,31.130847,31.130847,-1.825359,-1.825359,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,9944,20.0,2011-03-13,Mkinga Distric Coun,NaN,DWE,39.172796,-4.765587,Tajiri,NaN,Pangani,Moa/Mwereme,Tanga,4,8,Mkinga,Moa,1.0,True,GeoData Consultants Ltd,VWC,Zingibali,True,2009.0,submersible,submersible,submersible,vwc,user-group,pay per bucket,per bucket,salty,salty,enough,enough,other,other,unknown,communal standpipe multiple,communal standpipe,1,39.172796,39.172796,-4.765587,-4.765587,NaN,NaN,1.0,1.0,2009.0,2009.0,20.0,20.0
6,19816,NaN,2012-10-01,Dwsp,NaN,DWSP,33.362410,-3.766365,Kwa Ngomho,NaN,Internal,Ishinabulandi,Shinyanga,17,3,Shinyanga Rural,Samuye,NaN,True,GeoData Consultants Ltd,VWC,NaN,True,NaN,swn 80,swn 80,handpump,vwc,user-group,never pay,never pay,soft,good,enough,enough,machine dbh,borehole,groundwater,hand pump,hand pump,1,33.362410,33.362410,-3.766365,-3.766365,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,54551,NaN,2012-10-09,Rwssp,NaN,DWE,32.620617,-4.226198,Tushirikiane,NaN,Lake Tanganyika,Nyawishi Center,Shinyanga,17,3,Kahama,Chambo,Na

In [25]:
# Add columns for mean and standard deviation of critical features based on region, ward, overall

divisions = ['region', 'ward']

for null_feature in null_features:
    data['_'.join([null_feature, 'mean', 'overall'])] = train[null_feature].mean()
    data['_'.join([null_feature, 'std', 'overall'])] = train[null_feature].std()
    for division in divisions:
        new_feature_name_mean = '_'.join([null_feature, 'mean', division])
        new_feature_name_std = '_'.join([null_feature, 'std', division])
        
        calcs_mean = train.groupby(division)[null_feature].mean()
        calcs_std = train.groupby(division)[null_feature].std()
        for value in train[division].unique() :
            data.loc[data[division]==value, new_feature_name_mean] = calcs_mean[value]
            data.loc[data[division]==value, new_feature_name_std] = calcs_std[value]

In [26]:
# 1st step: Impute missing values with random numbers generated by normal distribution based on mean, std by 'ward'
# 2nd step (only applied on remaining null values): Impute missing values with random numbers generated by normal distribution based on mean, std by 'region'
# 3rd step (only applied on remaining null values): Impute missing values with random numbers generated by normal distribution based on mean, std by 'overall'

divisions_total = ['ward', 'region', 'overall']

for null_feature in null_features:
    for division in divisions_total:
        data['_'.join([null_feature,'imp_normal'])] = data.apply(lambda row: np.random.normal(loc=row['_'.join([null_feature,'mean',division])], scale=row['_'.join([null_feature,'std',division])]) if math.isnan(row['_'.join([null_feature,'imp_normal'])]) else row['_'.join([null_feature,'imp_normal'])], axis=1)
        display('Missing values after imputation in {} by {}: {}'.format(null_feature, division, data['_'.join([null_feature,'imp_normal'])].isnull().sum()))

'Missing values after imputation in longitude by ward: 1812'

'Missing values after imputation in longitude by region: 0'

'Missing values after imputation in longitude by overall: 0'

'Missing values after imputation in latitude by ward: 1812'

'Missing values after imputation in latitude by region: 0'

...

'Missing values after imputation in construction_year by region: 14283'

'Missing values after imputation in construction_year by overall: 0'

'Missing values after imputation in amount_tsh by ward: 33824'

'Missing values after imputation in amount_tsh by region: 14385'

'Missing values after imputation in amount_tsh by overall: 0'

In [27]:
data

,id,amount_tsh,date_recorded,funder,gps_height,installer,longitude,latitude,wpt_name,num_private,basin,subvillage,region,region_code,district_code,lga,ward,population,public_meeting,recorded_by,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,payment_type,water_quality,quality_group,quantity,quantity_group,source,source_type,source_class,waterpoint_type,waterpoint_type_group,train,longitude_imp_normal,longitude_imp_random_choice,latitude_imp_normal,latitude_imp_random_choice,gps_height_imp_normal,gps_height_imp_random_choice,population_imp_normal,population_imp_random_choice,construction_year_imp_normal,construction_year_imp_random_choice,amount_tsh_imp_normal,amount_tsh_imp_random_choice,longitude_mean_overall,longitude_std_overall,longitude_mean_region,longitude_std_region,longitude_mean_ward,longitude_std_ward,latitude_mean_overall,latitude_std_overall,latitude_mean_region,latitude_std_region,latitude_mean_ward,latitude_std_ward,gps_height_mean_overall,gps_height_std_overall,gps_height_mean_region,gps_height_std_region,gps_height_mean_ward,gps_height_std_ward,population_mean_overall,population_std_overall,population_mean_region,population_std_region,population_mean_ward,population_std_ward,construction_year_mean_overall,construction_year_std_overall,construction_year_mean_region,construction_year_std_region,construction_year_mean_ward,construction_year_std_ward,amount_tsh_mean_overall,amount_tsh_std_overall,amount_tsh_mean_region,amount_tsh_std_region,amount_tsh_mean_ward,amount_tsh_std_ward
0,69572,6000.0,2011-03-14,Roman,1390.0,Roman,34.938093,-9.856322,none,NaN,Lake Nyasa,Mnyusi B,Iringa,11,5,Ludewa,Mundindi,109.0,True,GeoData Consultants Ltd,VWC,Roman,False,1999.0,gravity,gravity,gravity,vwc,user-group,pay annually,annually,soft,good,enough,enough,spring,spring,groundwater,communal standpipe,communal standpipe,1,34.938093,34.938093,-9.856322,-9.856322,1390.000000,1390.0,109.000000,109.0,1999.000000,1999.0,6000.000000,6000.0,35.149669,2.607428,34.895989,0.507484,34.936860,0.018817,-5.885572,2.809876,-8.907700,0.695036,-9.849742,0.057023,1018.860839,612.566092,1697.442010,357.935649,1429.972222,85.347185,281.087167,564.68766,120.883051,207.994290,131.500000,85.804262,1996.814686,12.472045,1997.441284,11.488838,1997.527778,3.037726,1062.351942,5409.34494,1855.434340,5829.131050,4500.000000,2350.811730
1,8776,NaN,2013-03-06,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Zahanati,NaN,Lake Victoria,Nyamara,Mara,20,2,Serengeti,Natta,280.0,NaN,GeoData Consultants Ltd,Other,NaN,True,2010.0,gravity,gravity,gravity,wug,user-group,never pay,never pay,soft,good,insufficient,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,1,34.698766,34.698766,-2.147466,-2.147466,1399.000000,1399.0,280.000000,280.0,2010.000000,2010.0,1060.073858,NaN,35.149669,2.607428,34.153940,0.392713,34.560169,0.111100,-5.885572,2.809876,-1.739903,0.270569,-2.069085,0.074345,1018.860839,612.566092,1341.625190,157.456723,1367.591549,54.440683,281.087167,564.68766,538.794312,831.300180,305.169014,322.052302,1996.814686,12.472045,1997.454918,11.537118,1998.057143,14.160453,1062.351942,5409.34494,584.158038,3256.133649,542.857143,450.396651
2,34310,25.0,2013-02-25,Lottery Club,686.0,World vision,37.460664,-3.821329,Kwa Mahundi,NaN,Pangani,Majengo,Manyara,21,4,Simanjiro,Ngorika,250.0,True,GeoData Consultants Ltd,VWC,Nyumba ya mungu pipe scheme,True,2009.0,gravity,gravity,gravity,vwc,user-group,pay per bucket,per bucket,soft,good,enough,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,1,37.460664,37.460664,-3.821329,-3.821329,686.000000,686.0,250.000000,250.0,2009.000000,2009.0,25.000000,25.0,35.149669,2.607428,35.932915,0.696129,37.453988,0.013616,-5.885572,2.809876,-4.286211,0.520436,-3.812915,0.015699,1018.860839,612.566092,1426.077701,352.826622,683.500000,2.718251,281.087167,564.68766,317.778269,

In [13]:
# Add columns with list of values in corresponding group of region and ward, respectively

for null_feature in null_features:
    overall_list = list(train[null_feature])
    overall_list = [x for x in overall_list if not math.isnan(x)]
    data['_'.join([null_feature, 'list', 'overall'])] = data.apply(lambda row: overall_list, axis=1)
    display(null_feature, 'overall list done')
    for division in divisions:
        feature_name = '_'.join([null_feature, 'list', division])
        lists = train.groupby(division)[null_feature].apply(list)
        data[feature_name] = data.apply(lambda row: list() if row[division] not in train[division].unique() else lists[row[division]], axis=1)
        data[feature_name] = data[feature_name].apply(lambda lst: [x for x in lst if not math.isnan(x)])
        data[feature_name] = data[feature_name].apply(lambda x: np.nan if not x else x)
        display(null_feature, division)

In [ ]:
# 1st step: Impute missing values with empirical distribution grouped by 'ward'
# 2nd step (only applied on remaining null values): Impute missing values with empirical distribution grouped by 'region'
# 3rd step (only applied on remaining null values): Impute missing values with empirical distribution grouped by 'overall'

for null_feature in null_features:
    for division in divisions_total:        
        #data['_'.join([null_feature,'imp_random_choice'])] = data.apply(lambda row: np.random.choice(a=row['_'.join([null_feature,'list',division])]) if math.isnan(row['_'.join([null_feature,'imp_random_choice'])]) else row['_'.join([null_feature,'imp_random_choice'])], axis=1)
        data['_'.join([null_feature,'imp_random_choice'])] = data.apply(lambda row: row['_'.join([null_feature,'imp_random_choice'])] if not np.isnan(row['_'.join([null_feature,'imp_random_choice'])]).any() else (np.random.choice(a=row['_'.join([null_feature,'list',division])]) if not np.isnan(row['_'.join([null_feature,'list',division])]).any() else np.nan), axis=1)
        display('Missing values after imputation in {} by {}: {}'.format(null_feature, division, data['_'.join([null_feature,'imp_random_choice'])].isnull().sum()))
        

In [28]:
# Add columns for median of critical integer features based on region, ward, overall

float_var.append('latitude')

for var in int_var:
    data['_'.join([var, 'median', 'overall'])] = train[var].median()
    for division in divisions:
        new_feature_name_median = '_'.join([var, 'median', division])
        calcs_median = train.groupby(division)[var].median()
        for value in train[division].unique() :
            data.loc[data[division]==value, new_feature_name_median] = calcs_median[value]

In [29]:
# 1st step: Impute missing values with mean and median by 'ward'
# 2nd step (only applied on remaining null values): Impute missing values with mean and median by 'region'
# 3rd step (only applied on remaining null values): Impute missing values with overall mean and median

for var in float_var:
    for division in divisions_total:
        data[var] = data.apply(lambda row: row['_'.join([var,'mean',division])] if math.isnan(row[var]) else row[var], axis=1)
        display('Missing values after imputation in {} by {}: {}'.format(var, division, data[var].isnull().sum()))

for var in int_var:
    for division in divisions_total:
        data[var] = data.apply(lambda row: row['_'.join([var,'median',division])] if math.isnan(row[var]) else row[var], axis=1)
        display('Missing values after imputation in {} by {}: {}'.format(var, division, data[var].isnull().sum()))
        

'Missing values after imputation in amount_tsh by ward: 31659'

'Missing values after imputation in amount_tsh by region: 14374'

'Missing values after imputation in amount_tsh by overall: 0'

'Missing values after imputation in longitude by ward: 1812'

'Missing values after imputation in longitude by region: 0'

...

'Missing values after imputation in num_private by region: 31786'

'Missing values after imputation in num_private by overall: 0'

'Missing values after imputation in construction_year by ward: 23474'

'Missing values after imputation in construction_year by region: 14283'

'Missing values after imputation in construction_year by overall: 0'

In [34]:
# Drop columns used for generation of random numbers

drop_columns = list()
measures = 'mean std list'.split()
for null_feature in null_features:
    for division in divisions_total:
        for measure in measures:
            drop_columns.append('_'.join([null_feature, measure, division]))
for var in int_var:
    for division in divisions_total:
        for measure in measures:
            drop_columns.append('_'.join([var, 'median', division]))
data.drop(columns=drop_columns, inplace=True)

In [ ]:
data.isnull().sum()

In [ ]:
# create new feature that gives information about operational time
    
data['date_recorded'] = pd.to_datetime(data['date_recorded'])
data['operation_years'] = data.date_recorded.dt.year - data.construction_year
data['operation_years'] = data['operation_years'].astype(int)

In [ ]:
# scale numeric features

#num_features=['latitude','longitude','operation_years','amount_tsh', 'gps_height', 'population']
#scaler = MinMaxScaler()

#for s in split:
    #s[num_features] = scaler.fit_transform(s[num_features])

In [ ]:
# drop redundant features and features that do not seem to have an impact

data.drop(['extraction_type_group','extraction_type_class','payment','quality_group','source_class','source_type','waterpoint_type_group','management_group','quantity_group','date_recorded','wpt_name','num_private','recorded_by'],axis=1,inplace=True)

In [ ]:
data['funder'].value_counts()
data['installer'].value_counts()
data['scheme_name'].value_counts()

In [ ]:
# reduce dimension of categorical variables

# funder
data = data.assign(count = data.groupby('funder')['funder'].transform('count'))\
.sort_values(by = ['count','funder'], ascending = [False,True])

data.loc[data['count'] < 1050, 'funder'] = 'Others'
data['funder'].replace(np.nan, 'Others', inplace=True)
del data['count']

# installer
data = data.assign(count = data.groupby('installer')['installer'].transform('count'))\
.sort_values(by = ['count','installer'], ascending = [False,True])

data.loc[data['count'] < 765, 'installer'] = 'Others'
data.loc[data['installer'] == '0', 'installer'] = 'Others'
data['installer'].replace(np.nan, 'Others', inplace=True)
del data['count']

# scheme_name
data = data.assign(count = data.groupby('scheme_name')['scheme_name'].transform('count'))\
.sort_values(by = ['count','scheme_name'], ascending = [False,True])

data.loc[data['count'] < 296, 'scheme_name'] = 'Others'
data.loc[data['scheme_name'] == '0', 'scheme_name'] = 'Others'
data['scheme_name'].replace(np.nan, 'Others', inplace=True)
del data['count']

In [ ]:
# factorize features for evaluations

#data['funder'] = pd.factorize(data['funder'])[0]
#data['installer'] = pd.factorize(data['installer'])[0]
#data['basin'] = pd.factorize(data['basin'])[0]
#data['subvillage'] = pd.factorize(data['subvillage'])[0]
#data['region'] = pd.factorize(data['region'])[0]
#data['lga'] = pd.factorize(data['lga'])[0]
#data['ward'] = pd.factorize(data['ward'])[0]
#data['scheme_management'] = pd.factorize(data['scheme_management'])[0]
#data['scheme_name'] = pd.factorize(data['scheme_name'])[0]
#data['extraction_type'] = pd.factorize(data['extraction_type'])[0]
#data['management'] = pd.factorize(data['management'])[0]
#data['payment_type'] = pd.factorize(data['payment_type'])[0]
#data['water_quality'] = pd.factorize(data['water_quality'])[0]
#data['quantity'] = pd.factorize(data['quantity'])[0]
#data['waterpoint_type'] = pd.factorize(data['waterpoint_type'])[0]
#data['permit'] = pd.factorize(data['permit'])[0]
#data['source'] = pd.factorize(data['source'])[0]

In [ ]:
data.head()

In [ ]:
train_df = data[data["train"] == 1]
test_df = data[data["train"] == 0]

train_df.drop(["train"], axis=1, inplace=True)
test_df.drop(["train"], axis=1, inplace=True)

In [ ]:
pd.DataFrame(train_df).to_csv("./Data/train_cleaned_distr-imp.csv", index=False)
pd.DataFrame(test_df).to_csv("./Data/test_cleaned_distr_imp.csv", index=False)